# Gibbs sampling for linear regression in Python

Kieran Campbell - kieranc [at] well.ox.ac.uk

### Bayesian linear regression

Here we are interested in Gibbs sampling for normal linear regression with one independent variable. We assume we have paired data $(y_i, x_i), i = 1, \ldots, N $. We wish to find the posterior distributions of the coefficients $\beta_0$ (the intercept), $\beta_1$ (the gradient) and of the precision $\tau$ which is the reciprocal of the variance. The model can be written as 

$$ y_i \sim \mathcal{N}(\beta_0 + \beta_1 x_i, 1 / \tau) $$

or equivalently

$$ y_i = \beta_0 + \beta_1 x_i + \epsilon, \; \; \epsilon \sim \mathcal{N}(0, 1 / \tau) $$

The likelihood for this model may be written as

$$ L(y_1, \ldots, y_N, x_1, \ldots, x_N | \beta_0, \beta_1, \tau) = \prod_{i = 1}^N \mathcal{N}(\beta_0 + \beta_1 x_i, 1 / \tau) $$

We also wish to place [conjugate priors](https://en.wikipedia.org/wiki/Conjugate_prior) on $\beta_0$, $\beta_1$ and $\tau$ for reasons that will become apparent later. For these we choose

$$ \beta_0 \sim \mathcal{N}(\mu_0, 1 / \tau_0) $$
$$ \beta_1 \sim \mathcal{N}(\mu, 1 / \tau_1) $$
$$ \tau \sim \text{Gamma}(\alpha, \beta) $$


### Gibbs sampling

[Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling) works as follows: suppose we have two parameters $\theta_1$ and $\theta_2$ and some data $x$. Our goal is to find the posterior distribution of $p(\theta_1, \theta_2 | x)$. To do this in a Gibbs sampling regime we need to work out the conditional distributions $p(\theta_1 | \theta_2, x)$ and $p(\theta_2 | \theta_1, x)$ (which is typically the hard part). The Gibbs updates then proceed as follows:

1. Pick some initial $\theta_2^{(i)}$.
2. Sample $\theta_1^{(i+1)} \sim p(\theta_1 | \theta_2^{(i)}, x)$
3. Sample $\theta_2^{(i+1)} \sim p(\theta_2 | \theta_1^{(i+1)}, x)$

Then increment $i$ and repeat $K$ times to draw $K$ samples. This is equivalent to sampling new values for a given variable *while holding all others constant*. The key thing to remember in Gibbs sampling is to always use the most recent parameter values for all samples (e.g. sample $\theta_2^{(i+1)} \sim p(\theta_2 | \theta_1^{(i+1)}, x)$ and not $\theta_2^{(i+1)} \sim p(\theta_2 | \theta_1^{(i)}, x)$ provided $\theta_1^{(i+1)}$ has already been sampled).

The massive advantage of Gibbs sampling over other MCMC methods (namely Metropolis-Hastings) is that no tuning parameters are required! The downside is the need of a fair bit of maths to derive the updates, which even then aren't always guaranteed to exist.

### Pythonic setup

First let's set ourselves up with python imports and functions so we can implement the functions as we derive them.

In [5]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

### Deriving a Gibbs sampler

The general approach to deriving an update for a variable is 

1. Write down the posterior conditional density in log-form
2. Throw away all terms that don't depend on the current sampling variable
3. Pretend this is the density for your variable of interest and all other variables are fixed. What distribution does the log-density remind you of?
4. That's your conditional sampling density!

We go through this for our three variables step by step below.

#### Updates for $\beta_0$

We're interested in finding

$$ p(\beta_0 | \beta_1, \tau, y, x) \propto p(y, x | \beta_0, \beta_1, \tau) p(\beta_0) $$

Note that $p(y, x | \beta_0, \beta_1, \tau)$ is just the likelihood from above and $p(\beta_0)$ is simply $\mathcal{N}(\mu_0, 1 / \tau_0)$. 

If a variable $x$ follows a normal distribution with mean $\mu$ and precision $\tau$ then the log-dependence on $x$ is $-\frac{\tau}{2}(x - \mu)^2 \propto -\frac{\tau}{2} x^2 + \tau \mu x$. So if we can force the log-posterior conditional density into a quadratic form then the coefficient of $x^2$ (where $x$ is the variable of interest) will be $\tau \mu$ and the coefficient of $x^2$ will be $-\frac{\tau}{2}$. 

Hence the log-dependence on $\beta_0$ is

$$ -\frac{\tau_0}{2}(\beta_0 - \mu_0)^2 - \frac{\tau}{2} \sum_{i=1}^N (y_i - \beta_0 - \beta_1 x_i)^2 $$

Although it's perhaps not obvious, this expression is quadratic in $\beta_0$, meaning the conditional sampling density for $\beta_0$ will also be normal. A bit of algebra (dropping all terms that don't involve $\beta_0$ takes us to

$$ -\frac{\tau_0}{2} \beta_0^2 +\tau_0 \mu_0 \beta_0 -\frac{\tau}{2} N \beta_0^2 
+ \tau \sum_i (y_i - \beta_1 x_i) \beta_0$$

In other words the coefficient of $\beta_0$ is $\tau_0 \mu_0 + \tau \sum_i (y_i - \beta_1 x_i) $ while the coefficient of $\beta_0^2$ is $-\frac{\tau_0}{2} -\frac{\tau}{2} N$. This implies the conditional sampling distribution of $\beta_0$ is

$$ \beta_0 | \beta_1, \tau, \tau_0, \mu_0, x, y \sim \mathcal{N}\left( \frac{\tau_0 \mu_0 + \tau \sum_i (y_i - \beta_1 x_i)}{\tau_0 + \tau N}, 1 / (\tau_0 + \tau N) \right) $$

Let's turn that into a python function:

In [14]:
def sample_beta_0(y, x, beta_1, tau, tau_0, mu_0):
    N = len(y)
    assert len(x) == N
    assert len(tau_0) == 1
    assert len(mu_0) == 1
    precision = tau_0 + tau * N
    mean = tau_0 * mu_0 + tau * np.sum(y - beta_1 * x)
    mean /= precision
    return np.random.normal(mean, 1 / np.sqrt(precision))

Sweet! Now back to the maths.

#### Update for $\beta_1$

Similarly to $\beta_0$, the dependence of the conditional log-posterior is given by

$$ -\frac{\tau_1}{2}(\beta_1 - \mu_1)^2 - \frac{\tau}{2} \sum_{i=1}^N (y_i - \beta_0 - \beta_1 x_i)^2 $$

which if we expand out and drop all terms that don't include $\beta_1$ we get

$$ -\frac{\tau_1}{2} \beta_1^2 +\tau_1 \mu_1 \beta_1 -\frac{\tau}{2} \sum_i x_i^2 \beta_1^2 
+ \tau \sum_i (y_i - \beta_0) x_i \beta_1$$

so the coefficient of $\beta_1$ is $\tau_1 \mu_1 + \tau  \sum_i (y_i - \beta_0) x_i$ while the coefficient of $\beta_1^2$ is  $-\frac{\tau_1}{2} -\frac{\tau}{2} \sum_i x_i^2$. Therefore the conditional sampling density of $\beta_1$ is 

$$ \beta_1 | \beta_0, \tau, \mu_1, \tau_1, x, y \sim \mathcal{N}\left( \frac{\tau_1 \mu_1 + \tau  \sum_i (y_i - \beta_0) x_i}{\tau_1 + \tau \sum_i x_i^2}, 1 / (\tau_1 + \tau \sum_i x_i^2) \right) $$

Let's turn that into a Python function too: